In [ ]:
import pandas as pd 
import json
import os
import gzip, json


In [4]:
curr_dir = os.getcwd()
data_dir = os.path.join(curr_dir, "match_data")

In [ ]:
with gzip.open(os.path.join(data_dir,"match_batch_8.json.gz"), "rt", encoding="utf-8") as f:
    data = json.load(f)

print(data[0])


{'type': 'MATCHED_GAME', 'platform': 'NA1', 'gameDuration': 1496, 'queue': 420, 'participants': [{'subteamPlacement': 0, 'playerAugment1': 0, 'missions': {'playerScore0': 0, 'playerScore1': 0, 'playerScore2': 0, 'playerScore3': 0, 'playerScore4': 0, 'playerScore5': 0, 'playerScore6': 0, 'playerScore7': 0, 'playerScore8': 0, 'playerScore9': 0, 'playerScore10': 0, 'playerScore11': 0}, 'teamEarlySurrendered': False, 'teamPosition': 'TOP', 'placement': 0, 'challenges': {'12AssistStreakCount': 0, 'HealFromMapSources': 0, 'InfernalScalePickup': 0, 'SWARM_DefeatAatrox': 0, 'SWARM_DefeatBriar': 0, 'SWARM_DefeatMiniBosses': 0, 'SWARM_EvolveWeapon': 0, 'SWARM_Have3Passives': 0, 'SWARM_KillEnemy': 0, 'SWARM_PickupGold': 0, 'SWARM_ReachLevel50': 0, 'SWARM_Survive15Min': 0, 'SWARM_WinWith5EvolvedWeapons': 0, 'abilityUses': 376, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestIn

In [ ]:
def preprocess_match(data):
    res = {}
    for team in data['teams']:
        res = res | {f"{team['side'].upper()}_{participant['individualPosition']}": participant['championName'] for participant in team['participants']}
        res["RED_WINNER"] = team['isWinner'] if team['side'] == 'red' else not team['isWinner']

    return res

def build_dataset():
    res = []
    for i in range(2, 14):  # assuming you have 12 batches numbered from 2 to 13 in folder caled "match_data", which can be extracted from "api_fetching.ipynb"
        print("Processing batch ", i)
        with gzip.open(os.path.join(data_dir,f"match_batch_{i}.json.gz"), "rt", encoding="utf-8") as f:
            batched_data = json.load(f)
            res.extend(preprocess_match(data) for data in batched_data)
    return pd.DataFrame(res)


In [6]:
df = build_dataset()

Processing batch  2
Processing batch  3
Processing batch  4
Processing batch  5
Processing batch  6
Processing batch  7
Processing batch  8
Processing batch  9
Processing batch  10
Processing batch  11
Processing batch  12
Processing batch  13


In [7]:
df.to_csv(os.path.join(data_dir, "preprocessed_matches.csv"), index=False)